# XGBoost Classifier (Gradient-Boosting-Verfahren)

Imports

In [22]:
import time
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import numpy as np
import joblib

import xgboost as xgb

Config for XGBoost

In [23]:
pre_dispatch_jobs = 12
useGpu = False

Load data and labels into dataframes

In [24]:
df = pd.read_csv('../data/16_lucas_organic_carbon_training_and_test_data.csv')
labels = pd.read_csv('../data/16_lucas_organic_carbon_target.csv')

In [25]:
df.describe()

,500.0,502.0,504.0,506.0,508.0,510.0,512.0,514.0,516.0,518.0,...,2480.0,2482.0,2484.0,2486.0,2488.0,2490.0,2492.0,2494.0,2496.0,2498.0
count,1951.000000,1951.000000,1951.000000,1951.000000,1951.000000,1951.000000,1951.000000,1951.000000,1951.000000,1951.000000,...,1951.000000,1951.000000,1951.000000,1951.000000,1951.000000,1951.000000,1951.000000,1951.000000,1951.000000,1951.000000
mean,0.000272,0.000279,0.000286,0.000292,0.000299,0.000306,0.000313,0.000319,0.000324,0.000329,...,-0.000154,-0.000148,-0.000140,-0.000131,-0.000120,-0.000109,-0.000097,-0.000085,-0.000073,-0.000061
std,0.000124,0.000126,0.000129,0.000132,0.000134,0.000137,0.000139,0.000142,0.000144,0.000146,...,0.000059,0.000058,0.000057,0.000055,0.000053,0.000050,0.000048,0.000046,0.000044,0.000043
min,0.000016,0.000017,0.000018,0.000019,0.000021,0.000022,0.000023,0.000024,0.000025,0.000026,...,-0.000485,-0.000466,-0.000511,-0.000544,-0.000563,-0.000566,-0.000563,-0.000559,-0.000556,-0.000553
25%,0.000179,0.000184,0.000189,0.000194,0.000199,0.000203,0.000207,0.000212,0.000216,0.000219,...,-0.000185,-0.000179,-0.000170,-0.000160,-0.000148,-0.000136,-0.000122,-0.000109,-0.000097,-0.000083
50%,0.000266,0.000273,0.000280,0.000287,0.000294,0.000300,0.000307,0.000312,0.000318,0.000323,...,-0.000146,-0.000141,-0.000133,-0.000123,-0.000113,-0.000102,-0.000090,-0.000078,-0.000067,-0.000054
75%,0.000356,0.000365,0.000374,0.000382,0.000390,0.000398,0.000407,0.000414,0.000421,0.000428,...,-0.000114,-0.000109,-0.000102,-0.000093,-0.000083,-0.000073,-0.000063,-0.000053,-0.000042,-0.000031
max,0.000777,0.000788,0.000799,0.000809,0.000820,0.000831,0.000842,0.000851,0.000858,0.000865,...,-0.000022,-0.000019,-0.000015,-0.000010,-0.000005,-0.000001,0.000009,0.000023,0.000039,0.000055


Create Scaler and scala data

In [26]:
# TODO: Maybe try a z-transfromtion stead of the standardscaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_data, columns=df.columns)
scaled_df

,500.0,502.0,504.0,506.0,508.0,510.0,512.0,514.0,516.0,518.0,...,2480.0,2482.0,2484.0,2486.0,2488.0,2490.0,2492.0,2494.0,2496.0,2498.0
0,0.033550,0.030923,0.028393,0.025957,0.023611,0.021351,0.020134,0.019600,0.020371,0.022724,...,0.275892,0.280295,0.287538,0.297802,0.298202,0.276732,0.244755,0.207759,0.165672,0.118863
1,0.228730,0.240699,0.252153,0.263118,0.273620,0.283682,0.292945,0.302699,0.312571,0.322177,...,-0.761572,-0.805638,-0.843667,-0.865928,-0.860229,-0.825640,-0.773770,-0.711011,-0.636492,-0.550152
2,0.931905,0.920007,0.908408,0.897107,0.886101,0.875386,0.865490,0.855364,0.845065,0.834877,...,0.205335,0.234949,0.253837,0.257953,0.247090,0.213483,0.155301,0.090107,0.018342,-0.058809
3,0.723173,0.725128,0.726874,0.728428,0.729809,0.731031,0.730880,0.729221,0.725947,0.720612,...,0.307128,0.351822,0.390977,0.413210,0.409331,0.376629,0.321965,0.259427,0.189080,0.111728
4,0.685617,0.677358,0.669300,0.661443,0.653785,0.646324,0.638793,0.630961,0.623076,0.615126,...,0.095192,0.106626,0.116113,0.119311,0.122850,0.137148,0.148715,0.160365,0.171653,0.182027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,-0.634850,-0.617595,-0.600906,-0.584765,-0.569154,-0.554056,-0.538888,-0.523103,-0.506631,-0.489778,...,0.970707,0.944959,0.923588,0.898918,0.867846,0.832824,0.794746,0.747019,0.688553,0.618903
1947,0.601779,0.591457,0.581432,0.571699,0.562252,0.553084,0.542656,0.530403,0.515762,0.498423,...,1.470333,1.435445,1.394717,1.350529,1.304687,1.259562,1.199300,1.124194,1.032635,0.924024
1948,-0.386278,-0.373224,-0.360616,-0.348438,-0.336674,-0.325309,-0.311088,-0.292803,-0.269534,-0.240856,...,-1.941530,-1.899356,-1.845980,-1.782353,-1.707989,-1.618095,-1.506433,-1.372492,-1.214715,-1.033249
1949,0.268080,0.248295,0.229244,0.210896,0.193222,0.176193,0.159145,0.142823,0.127460,0.112641,...,-0.570091,-0.495802,-0.408980,-0.303210,-0.180018,-0.049235,0.095414,0.253978,0.424406,0.602904


Use lable encoder to transform high, low, etc to 1, 2, 3, 4, 5

In [27]:
label_encoder = LabelEncoder()
labels.x = label_encoder.fit_transform(labels.x)

Split Dataset into Training- and Testset

In [28]:
X_train, X_test, y_train, y_test = train_test_split(scaled_data, labels, test_size=0.2, random_state=42)

Create XGBoost Classifier

In [29]:
if useGpu:
    clf = xgb.XGBClassifier(learning_rate=0.02, n_estimators=10, objective='multi:softmax',
                            num_class=len(pd.unique(labels.x)), tree_method = "hist", device = "cuda")
else:
    clf = xgb.XGBClassifier(learning_rate=0.02, n_estimators=10, objective='multi:softmax',
                            num_class=len(pd.unique(labels.x)))
    
clf

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=10, n_jobs=None, num_class=5,
              num_parallel_tree=None, ...)

Create Parametergrid for Gridsearch / Randomsearch

In [30]:
param_grid = {
    'subsample': [0.6, 0.8],
    'colsample_bytree': [1.0],
    'gamma': [1],
    'min_child_weight': [5, 10],
    'max_depth': [3],
    'n_estimators': [int(x) for x in np.linspace(start = 550, stop = 650, num = 10)],
    'learning_rate': [0.1, 0.15]
}
param_grid

{'subsample': [0.6, 0.8],
 'colsample_bytree': [1.0],
 'gamma': [1],
 'min_child_weight': [5, 10],
 'max_depth': [3],
 'n_estimators': [550, 561, 572, 583, 594, 605, 616, 627, 638, 650],
 'learning_rate': [0.1, 0.15]}

Create Scoring method

In [31]:
scorer = make_scorer(f1_score, average='micro')

Run Grid/Randomsearch

In [ ]:
grid_search = GridSearchCV(clf, param_grid, cv=5, refit=True, verbose=3, n_jobs=-1,
                    pre_dispatch=pre_dispatch_jobs)

fit_time = time.perf_counter()
grid_search.fit(X_train, y_train)
fit_time = np.round(time.perf_counter() - fit_time)
print(f"Total Time to fit {fit_time}")

Fitting 5 folds for each of 80 candidates, totalling 400 fits


Extract the best classifier, its paramester and the score

In [40]:
best_clf = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

Run predictions on test-data and show model metrics

In [41]:
# Make predictions on the test set
y_pred = best_clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.5217391304347826
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.39      0.46        67
           1       0.42      0.56      0.48       106
           2       0.39      0.37      0.38       101
           3       0.62      0.73      0.67        22
           4       0.78      0.69      0.73        95

    accuracy                           0.52       391
   macro avg       0.55      0.55      0.54       391
weighted avg       0.54      0.52      0.52       391

Confusion Matrix:
[[26  8 24  8  1]
 [ 2 59 29  1 15]
 [14 46 37  1  3]
 [ 4  1  1 16  0]
 [ 0 26  3  0 66]]


Save model

In [42]:
joblib.dump(best_clf, '../new_models/xgboost.pkl')

['../new_models/xgboost.pkl']

Load model

In [43]:
model = joblib.load('../new_models/xgboost.pkl')
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=627, n_jobs=None, num_class=5,
              num_parallel_tree=None, ...)

Shaply

In [44]:
import shap

In [45]:
feature_names = df.columns.to_list()
explainer = shap.TreeExplainer(model, feature_names=feature_names)
explanation = explainer(df)

[23:44:49] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


In [46]:
file_path = f"../shapley-values/saved_values/xgboost-shapley_values.json"
joblib.dump(explanation, file_path)

['../shapley-values/saved_values/xgboost-shapley_values.json']